Это второй ноутбук по соревнованию. Здесь показано, как было получено лучшее предсказание. Из ноутбука со сравнением было видно, что модель XLM-Роберта оказалась лучшей для данной задачи, она одна уже пробивала проходной порог для домашней работы (0.9), однако, так как попадалось огромное количество ноутбуков с ансамблями, я решила также попробовать объединить предсказания нескольких моделей, основанных на лучшей модели и улучшить предсказание до 0.94. В данном ноутбуке показано как были обучены модели. Из-за особенностей kaggle kernel-only, этот ноутбук сначала запускался 3 раза, чтобы сохранить 3 разных предсказания. Затем уже сохраненные предсказания подгружались в кернел как сторонний инпут, чтобы объединить их с весами, и ноутбук запускался 4 раз, чтобы получить финальный сабмит как аутпут этого ноутбука и соответственно засабмитить его.

Данная часть полностью совпадает с той, что была приведена в ноутбуке со сравнением

In [0]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
import re

 Функция для предобработки для трансформера

In [0]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

Функция для создания модели 

In [0]:
def build_model(transformer, max_len=512, lr=1e-5, loss='binary_crossentropy'):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=lr), loss=loss, metrics=['accuracy', tf.keras.metrics.AUC()])
    
    
    return model

Настройка ТПУ

In [0]:

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)


Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


Гиперпараметры

In [0]:
AUTO = tf.data.experimental.AUTOTUNE
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
MODEL = 'jplu/tf-xlm-roberta-large'

In [0]:
# сам токенайзер
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Предобработка данных

In [0]:
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [0]:
train = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)
])

In [0]:
%%time 

x_train = regular_encode(train.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(valid.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test.content.values, tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values
y_valid = valid.toxic.values

CPU times: user 7min 5s, sys: 1.86 s, total: 7min 7s
Wall time: 7min 6s


Создание датасета и обучение

In [0]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)


In [0]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 192, 1024), (None 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________
CPU times: user 2min 1s, sys: 39.9 s, total: 2min 41s
Wall time: 2min 40s


In [0]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)


Train for 3404 steps, validate for 63 steps
Epoch 1/2


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


3404/3404 [==============================] - 1840s 540ms/step - loss: 0.0719 - accuracy: 0.9725 - auc: 0.9950 - val_loss: 0.2925 - val_accuracy: 0.8650 - val_auc: 0.9006
Epoch 2/2
3404/3404 [==============================] - 1633s 480ms/step - loss: 0.0510 - accuracy: 0.9794 - auc: 0.9976 - val_loss: 0.3137 - val_accuracy: 0.8648 - val_auc: 0.9029


In [0]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS
)

Train for 62 steps
Epoch 1/2


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


62/62 [==============================] - 123s 2s/step - loss: 0.2312 - accuracy: 0.8925 - auc: 0.9289
Epoch 2/2
62/62 [==============================] - 141s 2s/step - loss: 0.1468 - accuracy: 0.9360 - auc: 0.9731


Сохранение предсказания

In [0]:
# sub['toxic'] = model.predict(test_dataset, verbose=1)
# sub.to_csv('submission_4.csv', index=False)

Теперь еще немного дообучаем эту модель и сохраняем второе предсказание

In [0]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS
)

Train for 62 steps
Epoch 1/2


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


62/62 [==============================] - 124s 2s/step - loss: 0.0985 - accuracy: 0.9590 - auc: 0.9870
Epoch 2/2
62/62 [==============================] - 115s 2s/step - loss: 0.0613 - accuracy: 0.9782 - auc: 0.9951


In [0]:
# sub['toxic'] = model.predict(test_dataset, verbose=1)
# sub.to_csv('submission_6.csv', index=False)

Теперь добавляем новый элемент - обучение той же модели, но с добавлением некоторых особенностей обучения: другой лернинг рейт+ шедулер и дополнительная предобработка данных до использования токенайзера

In [0]:
test_data = test
train_data = train

Убираем мусор с помощью регулярных выражений

In [0]:
val = valid
train = train_data

def clean(text):
    text = text.fillna("fillna").str.lower()
    text = text.map(lambda x: re.sub('\\n',' ',str(x)))
    text = text.map(lambda x: re.sub("\[\[User.*",'',str(x)))
    text = text.map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
    text = text.map(lambda x: re.sub("\(http://.*?\s\(http://.*\)",'',str(x)))
    return text

val["comment_text"] = clean(val["comment_text"])
test_data["content"] = clean(test_data["content"])
train["comment_text"] = clean(train["comment_text"])

А также избавляемся от символов и сокращений

In [0]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}


def clean_text(x):
    x = str(x).replace("\n","")
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

Избавляемся от типичных ошибок, сокращений и тд

In [0]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


def clean_data(df, columns: list):
    for col in columns:

        df[col] = df[col].apply(lambda x: clean_text(x.lower())) 
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))
        df[col] = df[col].apply(lambda x: handle_contractions(x))  
        df[col] = df[col].apply(lambda x: fix_quote(x))   
    
    return df

In [0]:
input_columns = [
    'comment_text'   
]



train = clean_data(train, input_columns ) 
val = clean_data(val, input_columns )

In [0]:
input_columns = [
    'content'   
]
test_data = clean_data(test_data, input_columns )

del tokenizer

И только теперь подаем очищенный датасет в токенайзер модели

In [0]:
# сам токенайзер
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [0]:
x_train = regular_encode(train.comment_text.astype(str), 
                      tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(val.comment_text.astype(str).values, 
                      tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test_data.content.astype(str).values, 
                     tokenizer, maxlen=MAX_LEN)

y_valid = val.toxic.values
y_train = train.toxic.values


Создаем тф датасет

In [0]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

Создаем кастомный лосс (focal loss, который хорошо работает для несбалансированной классификации)

Создаем модель

In [0]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN, lr=3e-5)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 192, 1024), (None 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________
CPU times: user 28.4 s, sys: 26.7 s, total: 55.1 s
Wall time: 59.9 s


Также используем focal loss, которая хорошо подходит для несбалансированных выборок - как в данном случае. Создаем лернинг рэйт шэдулер

In [0]:
from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=.2):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

def build_lrfn(lr_start=0.000001, lr_max=0.000002, 
               lr_min=0.0000001, lr_rampup_epochs=7, 
               lr_sustain_epochs=0, lr_exp_decay=.87):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

In [0]:
lrfn = build_lrfn()


Создаем колбэк для обучения

In [0]:
from tensorflow.keras.callbacks import LearningRateScheduler
lr_callback = LearningRateScheduler(lrfn, verbose=1)
callback_list = [lr_callback]

Теперь также стандартный процесс обучения - данные строчки закомментированы, так как при попытке запустить все модели Roberta во время одного коммита, чтобы сохранить выходы ячеек, все слетает - Exhausted Resourses.  В общем-то тут стандартный процесс обучения - прикрепляю скрины.

In [0]:
# N_STEPS = x_train.shape[0] // BATCH_SIZE
# EPOCHS = 2
# train_history = model.fit(
#     train_dataset,
#     steps_per_epoch=N_STEPS,
#     validation_data=valid_dataset,
#     callbacks=callback_list,
#     epochs=EPOCHS
# )


In [0]:
# EPOCHS = 4
# n_steps = x_valid.shape[0] // BATCH_SIZE
# train_history_2 = model.fit(
#     valid_dataset.repeat(),
#     steps_per_epoch=n_steps,
#     callbacks=callback_list,
#     epochs= EPOCHS
# )


Процесс обучения на трейн:

![alt text](https://sun9-8.userapi.com/c857528/v857528479/205d89/VTZV4f0NJsE.jpg)

Обучение на валидационной части:

![alt text](https://sun9-61.userapi.com/c857528/v857528479/205d91/fj_Gb8vCcEk.jpg)

Сохранение предсказания

In [0]:
# sub['toxic'] = model.predict(test_dataset, verbose=1)
# sub.to_csv('submission_5.csv', index=False)


Теперь, когда уже все три предсказания сохранены как аутпуты ноутбука - подгружаем их в папку инпут, считываем, объединяем, умножая на коэффициенты и получаем итоговый сабмит.

In [0]:
sub1 = pd.read_csv("/kaggle/input/submission-roberta4/submission_4.csv")
sub2 = pd.read_csv("/kaggle/input/submission-roberta5/submission_5.csv")
sub3 = pd.read_csv("/kaggle/input/submissionroberta6/submission_6.csv")

In [0]:
sub1['toxic'] = sub1['toxic']*0.3 + sub2['toxic']*0.5 + sub3['toxic']*0.2

Сохраняем его и посылаем в кэггл

In [0]:
sub1.to_csv('submission.csv', index=False)

Финальный результат:

![alt text](https://sun9-11.userapi.com/c854120/v854120403/236734/N61N0rJ4Bxs.jpg)

Таким образом, объединение предсказаний нескольких моделей позволяет несколько улучшить качество

references:

опять же почти все ноутбуки с кэггл по соревнованию и еще отдельно стоит отметить вот этот чудесный кернел, откуда были взяты функция для предобработки данных и любопытная focal loss: https://www.kaggle.com/mobassir/understanding-cross-lingual-models